In [1]:
print("Hello world")

import os
import sys
import numpy as np
import pandas as pd
import math
import json
from tqdm import tqdm
from natsort import natsorted
import glob
import gc
import random

import torch

sys.path.append("/home/maxihuber/eeg-foundation")
from src.utils.preloading.utils import load_edf_to_dataframe

Hello world


In [2]:
erp = {
    "task_name": "ERP",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/erp/new_erp.json",
    "out_dim": 5,
    "outputs": set(
        [
            "Participant is in resting state",
            "with event-related potential",
            "Participant is in interval between two flashes",
            "without event-related potential",
            "Participant keeps closing eyes",
        ]
    ),
}

errp = {
    "task_name": "ERRP",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/erp/errp_all.json",
    "out_dim": 7,
    "outputs": set(
        [
            "Target is located in the right",
            "without error-related potential",
            "The cursor moves to the left",
            "The feedback consisted in the selected item is presented on the screen",
            "The cursor moves to the right",
            "with error-related potential",
            "Target is located in the left",
        ]
    ),
}

with open(errp["json_path"], "r") as f:
    train_test_data = json.load(f)

outputs = set([sample["label"] for sample in train_test_data["train"]])
print(outputs)

{'Target is located in the right', 'without error-related potential', 'The cursor moves to the left', 'with error-related potential', 'The feedback consisted in the selected item is presented on the screen', 'The cursor moves to the right', 'Target is located in the left'}


In [8]:
########################################################################################################################
# TUAB and Epilepsy

yc_class = {
    "class_name": "YC",
    "time_col": "Time in Seconds",
    "prefix_filepath": "/itet-stor/maxihuber/deepeye_storage/foundation/tueg/edf",
    "load_mode": 2,
}

tuab = {
    "task_name": "TUAB",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/cli/tuab.json",
    "out_dim": 2,
}

epilepsy = {
    "task_name": "Epilepsy",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/cli/epilepsy.json",
    "out_dim": 2,
}

yc_tasks = [tuab, epilepsy]

########################################################################################################################
# Clinical JSONs

cli_class = {
    "class_name": "Clinical",
    "time_col": "Time in Seconds",
    "prefix_filepath": "/itet-stor/maxihuber/deepeye_storage/foundation_clinical_prepared/",
    "load_mode": 0,
}

age = {
    "task_name": "Age",
    "task_type": "Regression",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/age2.json",
    "out_dim": 1,
}

depression = {
    "task_name": "Depression",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/cli_depression.json",
    "out_dim": 2,
}

parkinsons = {
    "task_name": "Parkinsons",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/parkinsons2.json",
    "out_dim": 2,
}

schizophrenia = {
    "task_name": "Schizophrenia",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/cli_schizophrenia.json",
    "out_dim": 2,
}

sex = {
    "task_name": "Sex",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_clinical_tasks/sex2.json",
    "out_dim": 2,
}

cli_tasks = [age, depression, parkinsons, schizophrenia, sex]


########################################################################################################################
# Motor-Imagery JSONs

mi_class = {
    "class_name": "Motor Imagery",
    "time_col": "time in seconds",
    "prefix_filepath": "/itet-stor/maxihuber/deepeye_storage/foundation_prepared/",
    "load_mode": 0,
}

eye_open_closed = {
    "task_name": "EyeOpenClosed",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/eye_open_closed.json",
    "out_dim": 2,
    "outputs": set(["eye open", "eye closed"]),
    "short_mode": False,
}

eye_vh = {
    "task_name": "EyeVH",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/eye_vh.json",
    "out_dim": 2,
    "outputs": set(["vertical", "horizontal"]),
    "short_mode": False,
}

flexion_extension_imaginary = {
    "task_name": "FlexionExtensionImaginary",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/flexion_extension_imaginary.json",
    "out_dim": 2,
    "outputs": set(
        [
            "hand movement imagined elbow flexion",
            "hand movement imagined elbow extension",
        ]
    ),
    "short_mode": False,
}

flexion_extension_real = {
    "task_name": "FlexionExtensionReal",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/flexion_extension_real.json",
    "out_dim": 2,
    "outputs": set(["hand movement elbow extension", "hand movement elbow flexion"]),
    "short_mode": False,
}

grasp_imaginary = {
    "task_name": "GraspImaginary",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/grasp_imaginary.json",
    "out_dim": 2,
    "outputs": set(["imagined palmar grasp", "imagined lateral grasp"]),
    "short_mode": False,
}

grasp_real = {
    "task_name": "GraspReal",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/grasp_real.json",
    "out_dim": 2,
    "outputs": set(["movement palmar grasp", "movement lateral grasp"]),
    "short_mode": False,
}

lr_imaginary = {
    "task_name": "LRImaginary",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/lr_imaginary.json",
    "out_dim": 2,
    "outputs": set(["left hand imagined movement", "right hand imagined movement"]),
    "short_mode": True,
}

lr_real = {
    "task_name": "LRReal",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/lr_real.json",
    "out_dim": 2,
    "outputs": set(["right hand movement", "left hand movement"]),
    "short_mode": True,
}

mi_task_body_parts_imagined = {
    "task_name": "BodyPartsImagined",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/mi_task_imagined_body_parts.json",
    "out_dim": 4,
    "outputs": set(
        [
            "rest",
            "right hand imagined movement",
            "foot imagined movement",
            "left hand imagined movement",
            "tongue imagined movement",
        ]
    ),
    "short_mode": True,
}

mi_task_body_parts_real = {
    "task_name": "BodyPartsReal",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/mi_task_body_parts.json",
    "out_dim": 4,
    "outputs": set(
        ["rest", "right hand movement", "foot movement", "left hand movement"]
    ),
    "short_mode": True,
}

pronation_supination_imaginary = {
    "task_name": "PronationSupinationImaginary",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/pronation_supination_imaginary.json",
    "out_dim": 2,
    "outputs": set(["imagined supination", "imagined pronation"]),
    "short_mode": False,
}

pronation_supination_real = {
    "task_name": "PronationSupinationReal",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/pronation_supination_real.json",
    "out_dim": 2,
    "outputs": set(["movement supination", "movement pronation"]),
    "short_mode": False,
}

mi_tasks = [eye_open_closed, eye_vh, flexion_extension_imaginary, flexion_extension_real, 
            grasp_imaginary, grasp_real, lr_imaginary, lr_real,
            mi_task_body_parts_imagined, mi_task_body_parts_real,
            pronation_supination_imaginary, pronation_supination_real]

########################################################################################################################
# ERP JSONs

erp_class = {
    "class_name": "Error-Related Potential",
    "time_col": "time in seconds",
    "prefix_filepath": "/itet-stor/maxihuber/deepeye_storage/foundation_prepared/",
    "load_mode": 0,
}

erp = {
    "task_name": "ERP",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/erp/new_erp.json",
    "out_dim": 5,
    "outputs": set(
        [
            "Participant is in resting state",
            "with event-related potential",
            "Participant is in interval between two flashes",
            "without event-related potential",
            "Participant keeps closing eyes",
        ]
    ),
}

errp = {
    "task_name": "ERRP",
    "task_type": "Classification",
    "json_path": "/itet-stor/maxihuber/deepeye_storage/foundation_tasks/erp/errp_all.json",
    "out_dim": 7,
    "outputs": set(
        [
            "Target is located in the right",
            "without error-related potential",
            "The cursor moves to the left",
            "The feedback consisted in the selected item is presented on the screen",
            "The cursor moves to the right",
            "with error-related potential",
            "Target is located in the left",
        ]
    ),
}

erp_tasks = [erp, errp]

########################################################################################################################
# EyeNet JSONs

eye_class = {
    "class_name": "EyeNet",
    "time_col": "time",
    "prefix_filepath": "/itet-stor/maxihuber/deepeye_storage/foundation_prepared/",
    "load_mode": 1,
}

eye_dir_amp = {
    "task_name": "EyeNetDirectionAmp",
    "task_type": "Regression",
    "json_path": [
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Direction_Amp_train.json",
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Direction_Amp_val.json",
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Direction_Amp_test.json",
    ],
    "out_dim": 1,
}

eye_dir_ang = {
    "task_name": "EyeNetDirectionAng",
    "task_type": "Regression",
    "json_path": [
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Direction_Ang_train.json",
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Direction_Ang_val.json",
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Direction_Ang_test.json",
    ],
    "out_dim": 1,
}

eye_lr = {
    "task_name": "EyeNetLR",
    "task_type": "Classification",
    "json_path": [
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_LR_train.json",
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_LR_val.json",
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_LR_test.json",
    ],
    "out_dim": 2,
}

eye_position = {
    "task_name": "EyeNetPosition",
    "task_type": "Regression",
    "json_path": [
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Position_train.json",
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Position_val.json",
        "/itet-stor/maxihuber/deepeye_storage/eegeyenet_tasks/EEGEyeNet_Position_test.json",
    ],
    "out_dim": 2,
}

eye_tasks = [eye_dir_amp, eye_dir_ang, eye_lr, eye_position]

classes = {
    "YC": [yc_class, yc_tasks],
    "Clinical": [cli_class, cli_tasks], 
    "MI": [mi_class, mi_tasks],
    "ERP": [erp_class, erp_tasks], 
    "EyeNet": [eye_class, eye_tasks],
}





In [15]:
import shutil

classes_to_copy = {
    "YC": [yc_class, yc_tasks],
    "MI": [mi_class, mi_tasks],
    "ERP": [erp_class, erp_tasks], 
}

base_path = '/itet-stor/maxihuber/net_scratch/finetune_files/'

for class_name, [class_dict, tasks] in classes_to_copy.items():
    for task in tasks:
        json_path = task["json_path"]
        json_path_new = os.path.join(base_path, os.path.basename(json_path))
        print(json_path_new)
        
        # Copy the file from json_path to json_path_new
        shutil.copy(json_path, json_path_new)
        print(f'Copied {json_path} to {json_path_new}')

/itet-stor/maxihuber/net_scratch/finetune_files/tuab.json
Copied /itet-stor/maxihuber/deepeye_storage/foundation_tasks/cli/tuab.json to /itet-stor/maxihuber/net_scratch/finetune_files/tuab.json
/itet-stor/maxihuber/net_scratch/finetune_files/epilepsy.json
Copied /itet-stor/maxihuber/deepeye_storage/foundation_tasks/cli/epilepsy.json to /itet-stor/maxihuber/net_scratch/finetune_files/epilepsy.json
/itet-stor/maxihuber/net_scratch/finetune_files/eye_open_closed.json
Copied /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/eye_open_closed.json to /itet-stor/maxihuber/net_scratch/finetune_files/eye_open_closed.json
/itet-stor/maxihuber/net_scratch/finetune_files/eye_vh.json
Copied /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/eye_vh.json to /itet-stor/maxihuber/net_scratch/finetune_files/eye_vh.json
/itet-stor/maxihuber/net_scratch/finetune_files/flexion_extension_imaginary.json
Copied /itet-stor/maxihuber/deepeye_storage/foundation_tasks/mi/flexion_extension_imaginary.js

In [4]:
def load_index0(data_index_path):
    with open(data_index_path, "r") as f:
        train_test_dict = json.load(f)
    train_samples = train_test_dict["train"]
    test_samples = train_test_dict["test"]
    return train_samples, test_samples

def load_index1(data_index_paths):
    all_samples = []
    for data_index_path in data_index_paths:
        with open(data_index_path, "r") as f:
            subset_dict = json.load(f)
        all_samples.append(list(subset_dict.values())[0])
    return all_samples[0], all_samples[1], all_samples[2]

def get_generic_channel_name(channel_name):
    channel_name = channel_name.lower()
    # Remove "eeg " prefix if present
    if channel_name.startswith("eeg "):
        channel_name = channel_name[4:]
    # Simplify names with a dash and check if it ends with "-"
    if "-" in channel_name:
        if channel_name.endswith("-"):
            return "None"
        return channel_name.split("-")[0]
    return channel_name

def get_node_index(index_patterns):
    index_paths = []
    for pattern in index_patterns:  # regex the index_patterns
        index_paths.extend(glob.glob(pattern))
    num_trials = 0
    trial_info_index = {}
    for index_path in index_paths:
        with open(index_path, "r") as f:
            new_trial_info_index = json.load(f)
            for trial_info in new_trial_info_index.values():
                trial_info_index[num_trials] = trial_info
                num_trials += 1
    print(f"[get_node_index] # Trials = {num_trials}", file=sys.stderr)
    return trial_info_index

def get_full_paths(input_files, prefix_filepath):
    adjusted_files = []
    for file in input_files:
        if file in filename_to_nodepath:
            adjusted_files.append(filename_to_nodepath[file])
        else:
            file = prefix_filepath + file if "/itet-stor" not in file else file.replace("/itet-stor/kard", "/itet-stor/maxihuber")
            adjusted_files.append(file)
    return adjusted_files

def get_size(data_index):
    try:
        sz = 0
        for sample in tqdm(data_index, desc="Estimating file size", position=0, leave=True):
            input_files = get_full_paths(sample["input"], prefix_filepath)
            # get file size in bytes combined for all input_files (raw filepaths)
            sz += sum([os.path.getsize(file) for file in input_files])
    except Exception as e:
            print(f"Failed to process sample: {sample}. Error: {e}", file=sys.stderr)
    return sz

def get_index_size(data_index, prefix_filepath):
    sz = 0
    for sample in data_index:
        try:
            input_files = get_full_paths(sample["input"], prefix_filepath)
            sz += sum([os.path.getsize(file) for file in input_files])
        except Exception as e:
            print(f"Failed to process sample: {sample}. Error: {e}", file=sys.stderr)
    return sz
    

def format_size(size_bytes):
    # Helper function to convert bytes to MB, GB, etc.
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return f"{s} {size_name[i]}"

def get_channels(data_index, task_channels):
    files_channels = []
    channel_set = set()
    for sample in tqdm(data_index, desc="Getting channels", position=0, leave=True):
        input_files = get_full_paths(sample["input"])
        file = input_files[0]
        if load_mode == 2:
            df = load_edf_to_dataframe(file)
        else:
            df = pd.DataFrame()
            with open(file, "rb") as f:
                df_new = pd.read_pickle(f)
                df = pd.concat([df, df_new], axis=0)
        channels = natsorted([get_generic_channel_name(ch) for ch in df.columns])
        files_channels.append(channels)
        # channel_set.update(channels)
        del df
        break
    return files_channels

def load_file_data(data_index, task_channels):
    num_samples = 0
    data = {}
    outputs = {}
    srs = {}
    durs = {}
    channels = {}
    datasets = {}
    failed_samples = []

    for sample in tqdm(data_index, desc="Loading data", position=0, leave=True):
        try:
            # Load and concatenate dataframe
            input_files = get_full_paths(sample["input"])

            if load_mode == 2:
                df = load_edf_to_dataframe(file)
            else:
                df = pd.DataFrame()
                for file in input_files:
                    if load_mode != 1:
                        file = prefix_filepath + file
                    else:
                        file = file.replace("/itet-stor/kard", "/itet-stor/maxihuber")                       
                    with open(file, "rb") as f:
                        df_new = pd.read_pickle(f)
                        df = pd.concat([df, df_new], axis=0)

            start = int(sample["start"])
            length = int(sample["length"]) if "length" in sample else int(sample["end"])
            if load_mode != 1:
                df = df.iloc[start:length, :]
                if short_mode:
                    df = df.iloc[: int(len(df) * 0.5), :]
            else:
                df = df.loc[start : start + length, :]

            # Add metadata
            if len(df) <= 1:
                assert False
            sr = int(
                1 / float(float(df[time_col].iloc[1]) - float(df[time_col].iloc[0]))
            )
            if load_mode != 1:
                outputs[num_samples] = (
                    sample["output"] if "output" in sample else sample["label"]
                )
            else:
                if task_name == "EyeNetPosition":
                    outputs[num_samples] = list(sample["output"].values())
                else:
                    outputs[num_samples] = list(sample["output"].values())[0]
            srs[num_samples] = sr
            durs[num_samples] = len(df) / sr
            channels[num_samples] = sorted(list(set(df.columns) & set(task_channels)), key=lambda x: list(task_channels).index(x))
            df = df[channels[num_samples]].astype(float)
            signals = torch.tensor(df.to_numpy(), dtype=torch.float32).T
            data[num_samples] = signals
            num_samples += 1

        except Exception as e:
            print(f"Failed to process sample: {sample}. Error: {e}", file=sys.stderr)
            failed_samples.append(sample)

    return data, outputs, srs, durs, channels, datasets

In [5]:
tueg = '/itet-stor/maxihuber/deepeye_storage/index_files/full_tueg_index2.json'
pkl = '/itet-stor/maxihuber/deepeye_storage/index_files/full_pkl_index.json'

with open(tueg, 'r') as f:
    tueg = json.load(f)

with open(pkl, 'r') as f:
    pkl = json.load(f)

path_to_ie = {os.path.basename(ie["path"]): ie for ie in (tueg + pkl)}

task_channels = '/home/maxihuber/eeg-foundation/src/data/components/channels_to_id3.json'
with open(task_channels, 'r') as f:
    task_channels = set(json.load(f).keys())

index_patterns = ["/dev/shm/mae/index_*.json", "/scratch/mae/index_*.json"]
node_index = get_node_index(index_patterns=index_patterns)
filename_to_nodepath = {os.path.basename(ie["origin_path"]): ie["new_path"] for trial_idx, ie in node_index.items()}

[get_node_index] # Trials = 216556


In [ ]:
def load_and_fetch_channels(used_class, used_task, task_channels):
    if load_mode != 1:
        train_index, test_index = load_index0(json_path)
    else:
        train_index, val_index, test_index = load_index1(json_path)

    if load_mode == 0 or load_mode == 2:
        train_channels = get_channels(train_index, task_channels)
        test_channels = get_channels(test_index, task_channels)
        channels = train_channels[0] + test_channels[0]
    elif load_mode == 1:
        train_channels = get_channels(train_index, task_channels)
        val_channels = get_channels(val_index, task_channels)
        test_channels = get_channels(test_index, task_channels)
        channels = train_channels[0] + val_channels[0] + test_channels[0]

    print(f"Class: {class_name}")
    print(f"Task: {task_name}")

    with open(f"/home/maxihuber/eeg-foundation/src/finetune/v2/channels_per_task/{class_name}_{task_name}_channels.json", "w") as f:
        json.dump(channels, f)

    channels_intersect = list(natsorted(list(set(channels) & set(task_channels))))

    with open(f"/home/maxihuber/eeg-foundation/src/finetune/v2/channels_per_task/{class_name}_{task_name}_channels_intersect.json", "w") as f:
        json.dump(channels_intersect, f)

    #################################################################

for class_name, [used_class, used_tasks] in classes.items():
    for used_task in used_tasks:
        class_name = used_class["class_name"]
        time_col = used_class["time_col"]
        prefix_filepath = used_class["prefix_filepath"]
        load_mode = used_class["load_mode"]
        task_name = used_task["task_name"]
        task_type = used_task["task_type"]
        json_path = used_task["json_path"]
        out_dim = used_task["out_dim"]
        short_mode = used_task["short_mode"] if "short_mode" in used_task else False

        print(f"Getting data sizes for {class_name} - {task_name}")

        if load_mode != 1:
            train_index, test_index = load_index0(json_path)
            print(f"Train size: {format_size(get_size(train_index))}")
            print(f"Test size: {format_size(get_size(test_index))}")
        else:
            train_index, val_index, test_index = load_index1(json_path)
            print(f"Train size: {format_size(get_size(train_index))}")
            print(f"Val size: {format_size(get_size(val_index))}")
            print(f"Test size: {format_size(get_size(test_index))}")

In [ ]:
def load_file_data(data_index, task_channels):
    num_samples = 0
    data = {}
    outputs = {}
    srs = {}
    durs = {}
    channels = {}
    datasets = {}
    failed_samples = []

    for sample in tqdm(data_index, desc="Loading data", position=0, leave=True):
        try:
            # Load the data of this sample
            input_files = get_full_paths(sample["input"])

            if load_mode == 2:
                file = input_files[0]
                df = load_edf_to_dataframe(file)
            else:
                dataframes = [pd.read_pickle(file) for file in input_files]
                df = pd.concat(dataframes, axis=0)
            
            # Crop the data to the desired length
            start = int(sample["start"])
            length = int(sample["length"]) if "length" in sample else int(sample["end"])
            df.loc[start : start + length, :] if load_mode==1 else df.iloc[start:length, :]
            assert len(df) > 0, f"Empty dataframe for sample: {sample}"
   
            # Add labels
            if load_mode != 1:
                outputs[num_samples] = sample.get("output", sample.get("label"))
            else:
                outputs[num_samples] = list(sample["output"].values()) if task_name == "EyeNetPosition" else list(sample["output"].values())[0]
            
            # Add metadata
            sr = int(1 / (df[time_col].iloc[1] - df[time_col].iloc[0]))
            srs[num_samples] = sr
            durs[num_samples] = len(df) / sr
            
            channels[num_samples] = sorted(list(set(df.columns) & set(task_channels)), key=lambda x: list(task_channels).index(x))
            df = df[channels[num_samples]].astype(float)
            data[num_samples] = torch.tensor(df.to_numpy(), dtype=torch.float32).T

            num_samples += 1

            del df
            if num_samples % 100 == 0:
                gc.collect()

        except Exception as e:
            print(f"Failed to process sample: {sample}. Error: {e}", file=sys.stderr)
            failed_samples.append(sample)

    return data, outputs, srs, durs, channels

used_class = yc_class
used_task = tuab

class_name = used_class["class_name"]
time_col = used_class["time_col"]
prefix_filepath = used_class["prefix_filepath"]
load_mode = used_class["load_mode"]
task_name = used_task["task_name"]
task_type = used_task["task_type"]
json_path = used_task["json_path"]
out_dim = used_task["out_dim"]
short_mode = used_task["short_mode"] if "short_mode" in used_task else False

if load_mode != 1:
    train_index, test_index = load_index0(json_path)
else:
    train_index, val_index, test_index = load_index1(json_path)

if load_mode == 0 or load_mode == 2:
    print("=" * 10 + "Load train data" + "=" * 100)
    train_data, train_outputs, train_sr, train_dur, train_channels, train_datasets = (
        load_file_data(train_index, task_channels)
    )
    print("=" * 10 + "Load test data" + "=" * 100)
    test_data, test_outputs, test_sr, test_dur, test_channels, test_datasets = (
        load_file_data(test_index, task_channels)
    )
elif load_mode == 1:
    train_data, train_outputs, train_sr, train_dur, train_channels, train_datasets = (
        load_file_data(train_index, task_channels)
    )
    val_data, val_outputs, val_sr, val_dur, val_channels, val_datasets = load_file_data(
        val_index, task_channels
    )
    test_data, test_outputs, test_sr, test_dur, test_channels, test_datasets = (
        load_file_data(test_index, task_channels)
    )
else:
    pass

## Shorten Tasks

In [12]:
def get_sample_size(sample, prefix_filepath):
        input_files = get_full_paths(sample["input"], prefix_filepath)
        sz = sum([os.path.getsize(file) for file in input_files])
        return sz

def sample_data(outputs, index, max_size):
    """Sample data indices based on class balance or randomly, respecting a maximum size limit."""
    selected_indices = []
    current_size = 0
    if task_type == "Classification":
        # We shuffle the outputs for each class to ensure random selection within each class
        for output, indices in outputs.items():
            random.shuffle(indices)
        all_outputs = list(outputs.keys())
        while current_size < max_size and all_outputs:
            for output in all_outputs[:]:
                if outputs[output]:
                    idx = outputs[output].pop(0)
                    size = get_sample_size(index[idx], prefix_filepath)
                    if current_size + size <= max_size:
                        selected_indices.append(idx)
                        current_size += size
                    else:
                        break
                else:
                    all_outputs.remove(output)
    else:  # Regression
        all_indices = [idx for indices in outputs.values() for idx in indices]
        random.shuffle(all_indices)
        for idx in all_indices:
            size = get_sample_size(index[idx], prefix_filepath)
            if current_size + size <= max_size:
                selected_indices.append(idx)
                current_size += size
            else:
                break
    return selected_indices

classes_to_make_light = {
    # 'YC': [yc_class, yc_tasks],
    'Clinical': [cli_class, cli_tasks],
    # 'MI': [mi_class, mi_tasks],
    # 'ERP': [erp_class, erp_tasks],
}

for class_name, [used_class, used_tasks] in classes_to_make_light.items():
    for used_task in used_tasks:
        class_name = used_class["class_name"]
        time_col = used_class["time_col"]
        prefix_filepath = used_class["prefix_filepath"]
        load_mode = used_class["load_mode"]
        task_name = used_task["task_name"]
        task_type = used_task["task_type"]
        json_path = used_task["json_path"]
        out_dim = used_task["out_dim"]
        short_mode = used_task["short_mode"] if "short_mode" in used_task else False

        print(f"Getting output values for {class_name} - {task_name}")

        if load_mode != 1:
            train_index, test_index = load_index0(json_path)
        else:
            train_index, val_index, test_index = load_index1(json_path)
        
        # Get output classes and their indices in train_index
        def get_outputs(data_index, load_mode):
            outputs = {}
            for i, sample in enumerate(data_index):
                output_val = sample.get("output", sample.get("label")) if load_mode != 1 else list(sample["output"].values()) if task_name == "EyeNetPosition" else list(sample["output"].values())[0]
                if output_val in outputs:
                    outputs[output_val].append(i)
                else:
                    outputs[output_val] = [i]
            return outputs

        def print_output_distribution(train_outputs, test_outputs, task_type):
            if task_type == "Classification":
                for output in train_outputs.keys():
                    try:
                        print(f"Output: {output} - Train-size: {len(train_outputs[output])} | Test-size: {len(test_outputs[output])}")
                    except KeyError:
                        print(f"Output: {output} - Train-size: {len(train_outputs[output])} | Test-size: 0")

        print(f"Originally had {len(train_index)} train and {len(test_index)} test indices.")
        train_outputs = get_outputs(train_index, load_mode)
        test_outputs = get_outputs(test_index, load_mode)
        # k1 = 'with event-related potential' if task_name == 'ERP' else 'with error-related potential'
        # k2 = 'without event-related potential' if task_name == 'ERP' else 'without error-related potential'
        # train_outputs = {k1: train_outputs[k1], k2: train_outputs[k2]}
        # test_outputs = {k1: test_outputs[k1], k2: test_outputs[k2]}
        print_output_distribution(train_outputs, test_outputs, task_type)

        # now, for classification, sample an element from each class at a time, until you have reached 50GB
        #  -> this should ensure balanced classes, and if one class is exhausted, you stop
        # for regression, sample randomly without replacement until you have reached 50GB (indices list, shuffled, pick until 50GB)
        # to get the size of a sample, you can make use of the get_sample_size method
        # train_new: <=50G, test_new: <=50GB
        train_index_new = sample_data(train_outputs, train_index, 50 * 1024**3)  # 50GB
        train_index_new = [train_index[i] for i in train_index_new]
        test_index_new = sample_data(test_outputs, test_index, 50 * 1024**3)  # 50GB
        test_index_new = [test_index[i] for i in test_index_new]

        print(f"Downsampled to {len(train_index_new)} ({format_size(get_index_size(train_index_new, prefix_filepath))}) train and {len(test_index_new)} ({format_size(get_index_size(test_index_new, prefix_filepath))}) test indices.")
        train_outputs_new = get_outputs(train_index_new, load_mode)
        test_outputs_new = get_outputs(test_index_new, load_mode)
        print_output_distribution(train_outputs_new, test_outputs_new, task_type)

        # store
        train_test_dict_light = {"train": train_index_new, "test": test_index_new}
        path_prefix = '/itet-stor/maxihuber/net_scratch/finetune_files/'
        filename_light = os.path.basename(json_path)[:-5] + "_light.json"
        json_path_light = path_prefix + filename_light

        with open(json_path_light, "w") as f:
            json.dump(train_test_dict_light, f)
            print(f"Dumped to {json_path_light}!")
        
        print("=" * 100)

Getting output values for Clinical - Age
Originally had 565 train and 139 test indices.
Downsampled to 91 (49.92 GB) train and 106 (49.76 GB) test indices.
Dumped to /itet-stor/maxihuber/net_scratch/finetune_files/age2_light.json.
Getting output values for Clinical - Depression
Originally had 206 train and 33 test indices.
Output: No Depression - Train-size: 170 | Test-size: 24
Output: Depression - Train-size: 36 | Test-size: 9
Downsampled to 158 (49.98 GB) train and 33 (10.6 GB) test indices.
Output: No Depression - Train-size: 122 | Test-size: 24
Output: Depression - Train-size: 36 | Test-size: 9
Dumped to /itet-stor/maxihuber/net_scratch/finetune_files/cli_depression_light.json.
Getting output values for Clinical - Parkinsons
Originally had 363 train and 129 test indices.
Output: PD - Train-size: 250 | Test-size: 78
Output: No PD - Train-size: 113 | Test-size: 51
Downsampled to 77 (50.0 GB) train and 129 (49.84 GB) test indices.
Output: PD - Train-size: 40 | Test-size: 78
Output: No